# GRASS GIS 101: Visualizing GRASS Data with folium

In this part, we will learn some data visualization with folium.

In [1]:
import sys
import os
import subprocess
from pathlib import Path
from urllib.parse import urlencode
from IPython.display import display
import pandas as pd
import seaborn as sns

# Ask GRASS GIS where its Python packages are and add them to the path
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS python packages we need
import grass.script as gs
import grass.jupyter as gj

# Start GRASS in the recently created project
gj.init("./dix_park/PERMANENT");


### About `r.contour`
`r.contour` is a GRASS GIS module used to generate vector contour lines from a raster elevation map. It allows for visualizing elevation levels as isolines. Contours can help identify terrain features like ridges and valleys.

In [18]:
gs.run_command("r.contour",input="dsm", output="con_multiple_col", step=5, overwrite=True)

The command below displays the attribute table of the vector map `con_multiple_col`.

In [22]:
!v.db.select map=con_multiple_col

cat|level
1|75
2|80
3|85
4|90
5|95
6|100
7|105
8|110
9|115
10|120
11|125
12|130
13|135
14|140
15|145


The code below exports the GRASS GIS vector map `con_multiple_col` to a GeoJSON file in WGS84 projection using the `v.out.ogr` command. The exported file is saved to `/tmp/con_multiple_col.geojson`, which can be used for web-based visualization tools like Folium.


In [24]:
import folium

# Define your vector map name
vector_map = "con_multiple_col"

# Export the vector to GeoJSON (must be in WGS84)
geojson_path = f"/tmp/{vector_map}.geojson"
gs.run_command(
    "v.out.ogr",
    input=vector_map,
    output=geojson_path,
    format="GeoJSON",
    overwrite=True
)

### 🗺️ Visualizing Contour Lines with a Color Gradient in Folium

This block creates an interactive Folium map that visualizes vector contour lines from the `con_multiple_col` map using a continuous color gradient. It loads the GeoJSON data, extracts contour elevation levels, and maps them to colors using the `Spectral` colormap from Matplotlib. A `style_function` dynamically assigns colors to each feature based on its elevation level, helping users visually distinguish terrain variations across the map.


In [30]:
import json
import matplotlib.cm as cm
import matplotlib.colors as mcolors
# Create a Folium map centered around the vector map
m = folium.Map(location=[35.761168, -78.668271], zoom_start=13)

# Load the vector data
vect = gj.Vector("con_multiple_col")

with open(vect._filename, encoding="utf-8") as file:
    data = json.load(file)

# Extract all contour levels to determine the color scale range
levels = [feature["properties"]["level"] for feature in data["features"]]
min_level = min(levels)
max_level = max(levels)

# Create a color map
cmap = cm.get_cmap("Spectral")
norm = mcolors.Normalize(vmin=min_level, vmax=max_level)

# Define the style function
def style_function(feature):
    level = feature["properties"]["level"]
    rgba_color = cmap(norm(level))
    hex_color = mcolors.rgb2hex(rgba_color)
    return {
        'fillColor': hex_color,
        'color': hex_color,
        'weight': 2,
        'fillOpacity': 0.7
    }

# Add the GeoJSON vector to the map with the style function
folium.GeoJson(
    data,
    style_function=style_function
).add_to(m)

# Show the map
m


/tmp/ipykernel_75996/3566017438.py:19: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("Spectral")
